<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/followers/subscribers_posts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vk_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.4 MB/s eta 0:00:00


Сделать функцию скрывающую реальный id пользователя id->новый id

In [ ]:
import vk_api
from vk_api.exceptions import ApiError

In [ ]:
def get_only_original_posts(user_id, access_token, max_posts=1000):
    """
    Получает только посты, опубликованные самим пользователем (без репостов)

    :param user_id: ID пользователя (например, 123456789)
    :param access_token: токен VK API
    :param max_posts: ограничение на количество постов (по умолчанию 1000)
    :return: список оригинальных постов или None в случае ошибки
    """
    try:
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        original_posts = []
        offset = 0
        count = 100

        while len(original_posts) < max_posts:
            response = vk.wall.get(
                owner_id=user_id,
                count=count,
                offset=offset
            )

            posts = response.get('items', [])
            if not posts:
                break

            for post in posts:
                if 'copy_history' not in post:
                    original_posts.append(post)
                    if len(original_posts) >= max_posts:
                        break

            offset += count
            time.sleep(0.4)

        return original_posts

    except ApiError as e:
        print(f"Ошибка VK API: {e}")
        return None

In [ ]:
import vk_api
from vk_api.exceptions import ApiError
import time

def get_all_user_posts(user_id, access_token, max_posts=10000):
    """
    Получает ВСЕ посты пользователя (с пагинацией)

    :param user_id: ID пользователя (например, 123456789)
    :param access_token: токен VK API
    :param max_posts: ограничение на максимальное количество постов (по умолчанию 1000)
    :return: список всех постов или None в случае ошибки
    """
    try:
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        all_posts = []
        offset = 0
        count = 100

        while True:
            response = vk.wall.get(
                owner_id=user_id,
                count=count,
                offset=offset
            )

            posts = response['items']
            if not posts:
                break

            all_posts.extend(posts)
            offset += count

            if len(all_posts) >= max_posts:
                break

            time.sleep(0.4)

        return all_posts

    except ApiError as e:
        print(f"Ошибка VK API: {e}")
        return None

In [ ]:
USER_ID = 103249

all_posts = get_all_user_posts(USER_ID, ACCESS_TOKEN)

if all_posts:
    print(f"Всего постов: {len(all_posts)}")
    for i, post in enumerate(all_posts[:10], 1):
        print(f"\nПост #{i}:")
        print(f"Дата: {post['date']}")
        if 'text' in post:
            print(f"Текст: {post['text']}")
else:
    print("Не удалось получить посты. Возможно, профиль закрыт или токен неверный.")

In [ ]:
import vk_api
from vk_api.exceptions import ApiError
import time
import csv
from datetime import datetime

def get_original_posts_with_text(user_id, access_token, max_posts=1000):
    """
    Получает оригинальные посты пользователя (без репостов) и с текстом
    :param user_id: ID пользователя (например, 123456789)
    :param access_token: токен VK API
    :param max_posts: максимальное количество постов (по умолчанию 1000)
    :return: список постов или None в случае ошибки
    """
    try:
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        posts = []
        offset = 0
        count = 100

        while len(posts) < max_posts:
            response = vk.wall.get(
                owner_id=user_id,
                count=count,
                offset=offset
            )

            items = response.get('items', [])
            if not items:
                break

            for post in items:
                if 'copy_history' not in post and post.get('text'):
                    posts.append(post)
                    if len(posts) >= max_posts:
                        break

            offset += count
            time.sleep(0.4)

        return posts

    except ApiError as e:
        print(f"Ошибка VK API: {e}")
        return None

def save_to_csv(posts, filename="vk_posts.csv"):
    """
    Сохраняет посты в CSV-файл
    :param posts: список постов
    :param filename: имя файла для сохранения
    """
    if not posts:
        print("Нет постов для сохранения.")
        return

    with open(filename, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(["Дата", "Текст", "Ссылка"])

        for post in posts:
            post_date = datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d %H:%M:%S')
            post_text = post['text'].replace('\n', ' ').strip()
            post_url = f"https://vk.com/wall{post['owner_id']}_{post['id']}"

            writer.writerow([post_date, post_text, post_url])

    print(f"Данные сохранены в {filename}")

In [ ]:
posts = get_original_posts_with_text(USER_ID, ACCESS_TOKEN)

if posts:
    print(f"Найдено {len(posts)} постов.")
    save_to_csv(posts)
else:
    print("Не удалось получить посты.")

Найдено 1000 постов.
Данные сохранены в vk_posts.csv


In [ ]:
import vk_api
from vk_api.exceptions import ApiError
import time
import csv
from datetime import datetime
import pandas as pd

def get_original_posts_with_text(user_id, access_token, max_posts=1000):
    """Получает оригинальные посты пользователя с текстом"""
    try:
        vk_session = vk_api.VkApi(token=access_token)
        vk = vk_session.get_api()

        posts = []
        offset = 0
        count = 100

        while len(posts) < max_posts:
            try:
                response = vk.wall.get(
                    owner_id=user_id,
                    count=count,
                    offset=offset
                )

                items = response.get('items', [])
                if not items:
                    break

                for post in items:
                    if 'copy_history' not in post and post.get('text'):
                        posts.append({
                            'user_id': user_id, # удалить
                            'date': post['date'],
                            'text': post['text'],
                            'post_id': post['id']
                        })
                        if len(posts) >= max_posts:
                            break

                offset += count
                time.sleep(0.4)

            except ApiError as e:
                print(f"Ошибка для пользователя {user_id}: {e}")
                break

        return posts

    except Exception as e:
        print(f"Критическая ошибка для пользователя {user_id}: {e}")
        return []

def process_all_users(input_csv, output_csv, access_token, max_posts_per_user=100):
    """Обрабатывает всех пользователей из CSV"""
    try:
        followers = pd.read_csv(input_csv)
        user_ids = followers['id'].tolist()

        all_posts = []

        for user_id in user_ids:
            posts = get_original_posts_with_text(user_id, access_token, max_posts_per_user)
            all_posts.extend(posts)
            time.sleep(0.1)

        if all_posts:
            df = pd.DataFrame(all_posts)
            df['date'] = pd.to_datetime(df['date'], unit='s')
            df['url'] = df.apply(lambda x: f"https://vk.com/wall{x['user_id']}_{x['post_id']}", axis=1)
            df.to_csv(output_csv, index=False, encoding='utf-8', sep=';')
            print(f"Сохранено {len(df)} постов в {output_csv}")
        else:
            print("Не найдено ни одного поста")

    except Exception as e:
        print(f"Ошибка при обработке файла: {e}")


ACCESS_TOKEN = "you_acces_token_vk"
INPUT_FILE = "/content/baptist_followers.csv"
OUTPUT_FILE = "/content/baptists_posts.csv"

In [ ]:
import hashlib

def hide_user_id(real_id):
    hashed_id = hashlib.sha256(str(real_id).encode()).hexdigest()
    return hashed_id[:10]


real_id = 103249
new_id = hide_user_id(real_id)
print(f"Реальный ID: {real_id}, Новый ID: {new_id}")

In [ ]:
process_all_users(INPUT_FILE, OUTPUT_FILE, ACCESS_TOKEN)

Ошибка для пользователя 186754: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 224102: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 512781: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 899221: [30] This profile is private
Ошибка для пользователя 901457: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 1645365: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 2042325: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 2244152: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 2325480: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 2428973: [15] Access denied: user hid his wall from accessing from outside
Ошибка для пользователя 2531822: [15] Access denied:

In [ ]:
import vk_api
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
import seaborn as sns
from tqdm import tqdm

def analyze_activity(posts):
    """Анализ активности по временным периодам"""
    df = pd.DataFrame(posts)
    if df.empty:
        return None

    # Группировка по временным периодам
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['year_month'] = df['date'].dt.to_period('M')
    df['weekday'] = df['date'].dt.weekday
    df['hour'] = df['date'].dt.hour

    # Анализ по годам
    yearly = df.groupby('year').size().reset_index(name='count')

    # Анализ по месяцам
    monthly = df.groupby('year_month').size().reset_index(name='count')
    monthly['year_month'] = monthly['year_month'].astype(str)

    # Анализ по дням недели
    weekdays = df.groupby('weekday').size().reset_index(name='count')
    weekday_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
    weekdays['weekday_name'] = weekdays['weekday'].apply(lambda x: weekday_names[x])

    # Анализ по часам
    hourly = df.groupby('hour').size().reset_index(name='count')

    return {
        'yearly': yearly,
        'monthly': monthly,
        'weekdays': weekdays,
        'hourly': hourly,
        'total_posts': len(df)
    }

def visualize_activity(analysis, filename):
    """Визуализация результатов анализа"""
    plt.figure(figsize=(15, 10))

    # График по годам
    plt.subplot(2, 2, 1)
    sns.barplot(x='year', y='count', data=analysis['yearly'])
    plt.title('Активность по годам')

    # График по месяцам
    plt.subplot(2, 2, 2)
    sns.lineplot(x='year_month', y='count', data=analysis['monthly'])
    plt.title('Активность по месяцам')
    plt.xticks(rotation=45)

    # График по дням недели
    plt.subplot(2, 2, 3)
    sns.barplot(x='weekday_name', y='count', data=analysis['weekdays'], order=weekday_names)
    plt.title('Активность по дням недели')

    # График по часам
    plt.subplot(2, 2, 4)
    sns.barplot(x='hour', y='count', data=analysis['hourly'])
    plt.title('Активность по часам дня')

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

In [ ]:
plt.subplot(2, 2, 1)
sns.barplot(x='year', y='count', data=analysis['yearly'])
plt.title('Активность по годам')

In [ ]:
plt.subplot(2, 2, 2)
sns.lineplot(x='year_month', y='count', data=analysis['monthly'])
plt.title('Активность по месяцам')
plt.xticks(rotation=45)

In [ ]:
plt.subplot(2, 2, 3)
sns.barplot(x='weekday_name', y='count', data=analysis['weekdays'], order=weekday_names)
plt.title('Активность по дням недели')

In [ ]:
plt.subplot(2, 2, 4)
sns.barplot(x='hour', y='count', data=analysis['hourly'])
plt.title('Активность по часам дня')